### boor-system

In [11]:
import sys
sys.path.append('/Users/baza/dev/python-boorstat/')

# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# import plotly.plotly as py
# import plotly.graph_objs as go

import boorstat

### boor-title

Russian Writers CSV

### boor-short-content

[empty]

### boor-full-content

It was always interesting for me how different russian writers relate to each other.<br/>
For example, could Dostoyevky meet Gogol?<br/>
We are going to create csv with brief info like year of life for all russian writers.<br/>
I did not think too much and just parse <a href="https://en.wikipedia.org/wiki/List_of_Russian-language_writers">appropriate wiki page</a> – thanks to it:

In [12]:
import requests
from bs4 import BeautifulSoup

WIKI_RUS_WRITERS_URL = 'https://en.wikipedia.org/wiki/List_of_Russian-language_writers'

soup = BeautifulSoup(requests.get(WIKI_RUS_WRITERS_URL).content, 'lxml')

In [13]:
import re

all_lis = soup.find('div', id='mw-content-text').find_all('li')
lis = [li.text for li in all_lis if re.search(r'> \(', str(li))]

def extract_fields(li):
    name = li.partition('(')[0].strip()

    years = re.search(r'\((.+?\))', li).group(0)
    years = re.findall(r'[0-9]{4}', years)

    birth, death = '', ''
    if len(years) == 1:
        birth = years[0]
    elif len(years) == 2:
        birth, death = years
        
    descr = li.partition(')')[2]
    descr_parts = re.split(r', (?=[A-Z])', descr)
    descr = descr_parts[0].strip().strip(',.;').strip()

    works = ''
    if len(descr_parts) > 1:
        works = ','.join(descr_parts[1:])

    return name, birth, death, descr, works

lis = [extract_fields(li) for li in lis]
lis = [li for li in lis if not (li[1] == '' and li[2] == '')]

columns = ('name', 'birth_year', 'death_year', 'about', 'works')
separator = ';'

def save_to(s):
    s.write(';'.join(columns))
    s.write('\n')
    for li in lis:
        s.write(';'.join(li))
        s.write('\n')
        
from io import StringIO
csv = StringIO()
save_to(csv)

# # or local file
# with open('russian-writers.csv', 'w') as f:
#     save_to(f)

print(csv.getvalue()[:1000], '...')

name;birth_year;death_year;about;works
Alexander Ablesimov;1742;1783;opera librettist, poet, dramatist, satirist and journalist;
Fyodor Abramov;1920;1983;novelist and short story writer;Two Winters and Three Summers
Grigory Adamov;1886;1945;science fiction writer;The Mystery of the Two Oceans
Georgy Adamovich;1892;1972;poet, critic, memoirist, tanslator;
Alexander Afanasyev;1826;1871;folklorist who recorded and published over 600 Russian folktales and fairytales;Russian Fairy Tales
Alexander Afanasyev-Chuzhbinsky;1816;1875;poet, writer, ethnographer and translator;
Alexander Afinogenov;1904;1941;playwright;A Far Place
M. Ageyev;1898;1973;pseudonymous writer;Cocain Romance
Chinghiz Aitmatov;1928;2008;;Kyrgyz novelist and short story writer,Jamilya,The Day Lasts More Than a Hundred Years
David Aizman;1869;1922;;Russian-Jewish writer and playwright
Bella Akhmadulina;1937;2010;modern poet;The String
Anna Akhmatova;1889;1966;acmeist poet;Requiem
Ivan Aksakov;1823;1886;journalist, slavophile

<a href="https://raw.githubusercontent.com/boorstat/boorstat-files/master/lit/russian/russian-writers.csv">Russian Writers CSV</a><br/>
<a href="https://github.com/boorstat/boorstat-files/blob/master/lit/russian/russian-writers.csv">The same in github viewer</a>

In [14]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

RUS_WITERS_CSV = 'https://raw.githubusercontent.com/boorstat/boorstat-files/master/lit/russian/russian-writers.csv'

Getting csv as pandas dataframe:

In [23]:
df = pd.DataFrame.from_csv(StringIO(requests.get(RUS_WITERS_CSV).text), index_col=None, sep=';')
df = df[['name', 'birth_year', 'death_year']]
df.rename(columns={'name': 'Task', 'birth_year': 'Start', 'death_year': 'Finish'}, inplace=True)
df.head()

,Task,Start,Finish
0,Alexander Ablesimov,1742,1783.0
1,Fyodor Abramov,1920,1983.0
2,Grigory Adamov,1886,1945.0
3,Georgy Adamovich,1892,1972.0
4,Alexander Afanasyev,1826,1871.0


In [87]:
import plotly.figure_factory as FF
import math

RUS_WRITERS_FAVS = ['Dostoyevsky', 'Tolstoy', 'Gogol']

# df = [dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28'),
#       dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15'),
#       dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30')]
df = pd.DataFrame.from_csv(StringIO(requests.get(RUS_WITERS_CSV).text), index_col=None, sep=';')
df = df[['name', 'birth_year', 'death_year']]
df.rename(columns={'name': 'Task', 'birth_year': 'Start', 'death_year': 'Finish'}, inplace=True)
# df['Start'] = df[['Start']].apply(lambda v: print(str(v[0])), axis=1)
# df[:10]
                                  
df['Start'] = df[['Start']].apply(lambda v: '{}-01-01'.format(str(int(v))), axis=1)
df['Finish'] = df[['Finish']].apply(lambda v: '{}-12-31'.format(str(int(v)) if not math.isnan(v) else '2020'), axis=1)

df = df[df['Task'].map(lambda v: any([w in v for w in RUS_WRITERS_FAVS]))]
df.reset_index(inplace=True)

def _rename(name):
    parts = '<br>'.join(name.split())
    return '{name}{surname}'.format(
        name='{}<br>'.format(name[0]) if len(parts) > 1 else '',
        surname=parts[-1])

df['Task'] = df[['Task']].apply(lambda v: '<br>'.join(str(v[0]).split()), axis=1)

fig = FF.create_gantt(df, showgrid_x=True, showgrid_y=True)
py.iplot(fig, filename='russian-writers-years-of-life', world_readable=True)

Looks very exciting as for me:)<br/>
Time to plot Pushkin's Duels Histogram.

In [9]:
df[:10]

,name,birth_year,death_year,about,works
0,Alexander Ablesimov,1742,1783.0,"opera librettist, poet, dramatist, satirist an...",NaN
1,Fyodor Abramov,1920,1983.0,novelist and short story writer,Two Winters and Three Summers
2,Grigory Adamov,1886,1945.0,science fiction writer,The Mystery of the Two Oceans
3,Georgy Adamovich,1892,1972.0,"poet, critic, memoirist, tanslator",NaN
4,Alexander Afanasyev,1826,1871.0,folklorist who recorded and published over 600...,Russian Fairy Tales
5,Alexander Afanasyev-Chuzhbinsky,1816,1875.0,"poet, writer, ethnographer and translator",NaN
6,Alexander Afinogenov,1904,1941.0,playwright,A Far Place
7,M. Ageyev,1898,1973.0,pseudonymous writer,Cocain Romance
8,Chinghiz Aitmatov,1928,2008.0,NaN,"Kyrgyz novelist and short story writer,Jamilya..."
9,David Aizman,1869,1922.0,NaN,Russian-Jewish writer and playwright


In [38]:
life_years = {'start': 1799, 'end': 1837, 'size': 1}

nobody_shot_data = go.Histogram(
    x=df[(df['pushkin_shot'] == 0) & (df['opponent_shot'] == 0)]['year'],
    xbins=life_years,
    name='Nobody'
)

only_opponent_shot_data = go.Histogram(
    x=df[(df['pushkin_shot'] == 0) & (df['opponent_shot'] == 1)]['year'],
    xbins=life_years,
    name='Only opponent'
)

both_shot_data = go.Histogram(
    x=df[(df['pushkin_shot'] == 1) & (df['opponent_shot'] == 1)]['year'],
    xbins=life_years,
    name='Both shot'
)

data = [only_opponent_shot_data, nobody_shot_data, both_shot_data]
layout = go.Layout(barmode='stack', title="Pushkin's Duels Histogram")
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='pushkin-duels')